In [ ]:
from model import load_trpb_dataset, TrpBGrowthModel

Prior predictive check for the TrpB datasets.

In [3]:
data = load_trpb_dataset(
    "~/GitRepos/DMSStan/raw_data/trpb/3-site_merged_replicates/LibI/20230926/LibI_merged_AAs.csv"
)
model = TrpBGrowthModel(**data)
model.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'4abd38f0-d9f6-4f66-bff3-18733b6b6cf5': {'version…

# MAP

Now that we've selected our priors, we're ready to identify the MAP:

In [ ]:
map_ = model.approximate_map()
map_.plot_loss_curve()

# MCMC

We can now use Stan to sample from the posterior:

In [ ]:
# samples = model.mcmc(
#     output_dir="./tmp",
#     cpp_options={"STAN_THREADS": True},
# )

20:42:02 - cmdstanpy - INFO - compiling stan file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/tmp/model.stan to exe file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/tmp/model


20:42:18 - cmdstanpy - INFO - compiled model executable: /home/bwittmann/GitRepos/DMSStan/flip3/trpB/tmp/model
20:42:18 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

00:38:40 - cmdstanpy - INFO - CmdStan done processing.
00:38:40 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: model_model_namespace::log_prob: theta_tg0[1, 1][468] is -nan, but must be greater than or equal to 0.000000 (in 'model.stan', line 22, column 2 to column 48)
	Exception: model_model_namespace::log_prob: theta_tg0[1, 1][468] is -nan, but must be greater than or equal to 0.000000 (in 'model.stan', line 22, column 2 to column 48)
	Exception: model_model_namespace::log_prob: theta_tg0[1, 1][468] is -nan, but must be greater than or equal to 0.000000 (in 'model.stan', line 22, column 2 to column 48)
	Exception: model_model_namespace::log_prob: theta_tg0[1, 1][468] is -nan, but must be greater than or equal to 0.000000 (in 'model.stan', line 22, column 2 to column 48)
	Exception: model_model_namespace::log_prob: theta_tg0[1, 1][646] is -nan, but must be greater than or equal to 0.000000 (in 'model.stan', line 22, column 2 to column 48)
	Exception: model_model_n